In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path as path

companies_csv_path = 'companies.csv'
price_history_path = 'history_combined.csv'

In [2]:
companies = pd.read_csv(companies_csv_path)
history = pd.read_csv(price_history_path)
history

,Symbol,Date,Low,High,Open,Close,Volume
0,A,1999-11-18,28.612303,35.765381,32.546494,31.473534,62546380.0
1,A,1999-11-19,28.478184,30.758226,30.713518,28.880545,15234146.0
2,A,1999-11-22,28.657009,31.473534,29.551144,31.473534,6577870.0
3,A,1999-11-23,28.612303,31.205294,30.400572,28.612303,5975611.0
4,A,1999-11-24,28.612303,29.998213,28.701717,29.372318,4843231.0
...,...,...,...,...,...,...,...
6127889,ZTS,2017-11-06,68.749000,70.377000,68.819000,69.608000,3890518.0
6127890,ZTS,2017-11-07,68.045000,70.157000,69.628000,68.350000,5017394.0
6127891,ZTS,2017-11-08,67.760000,69.620000,68.000000,69.510000,3933647.0
6127892,ZTS,2017-11-09,68.960000,69.500000,69.160000,69.300000,2406718.0


In [3]:
# Sort and drop duplicates
history = history.sort_values(by=['Symbol', 'Date'])
history = history.drop_duplicates(subset=['Symbol', 'Date'])
history.reset_index(drop=True, inplace=True)
history

,Symbol,Date,Low,High,Open,Close,Volume
0,A,1999-11-18,28.612303,35.765381,32.546494,31.473534,62546380.0
1,A,1999-11-19,28.478184,30.758226,30.713518,28.880545,15234146.0
2,A,1999-11-22,28.657009,31.473534,29.551144,31.473534,6577870.0
3,A,1999-11-23,28.612303,31.205294,30.400572,28.612303,5975611.0
4,A,1999-11-24,28.612303,29.998213,28.701717,29.372318,4843231.0
...,...,...,...,...,...,...,...
3915011,ZTS,2022-12-06,152.089996,155.500000,154.220001,153.050003,1964800.0
3915012,ZTS,2022-12-07,149.380005,153.789993,152.960007,150.250000,2444100.0
3915013,ZTS,2022-12-08,149.199997,154.350006,150.529999,153.679993,2267500.0
3915014,ZTS,2022-12-09,152.740005,156.330002,153.940002,153.389999,3274900.0


In [4]:
# Rescale the volume column with log scaling
# Normalize the volume column
history['Volume'] = pd.to_numeric(history['Volume'])
history['VolumeScaledNormalized'] = history['Volume']
history['VolumeScaledNormalized'] += 1
history['VolumeScaledNormalized'] = np.log(history['VolumeScaledNormalized'])
history[['VolumeScaledNormalized']] -= history[['VolumeScaledNormalized']].min()
history[['VolumeScaledNormalized']] /= history[['VolumeScaledNormalized']].max()
history

,Symbol,Date,Low,High,Open,Close,Volume,VolumeScaledNormalized
0,A,1999-11-18,28.612303,35.765381,32.546494,31.473534,62546380.0,0.789849
1,A,1999-11-19,28.478184,30.758226,30.713518,28.880545,15234146.0,0.727706
2,A,1999-11-22,28.657009,31.473534,29.551144,31.473534,6577870.0,0.690754
3,A,1999-11-23,28.612303,31.205294,30.400572,28.612303,5975611.0,0.686529
4,A,1999-11-24,28.612303,29.998213,28.701717,29.372318,4843231.0,0.677284
...,...,...,...,...,...,...,...,...
3915011,ZTS,2022-12-06,152.089996,155.500000,154.220001,153.050003,1964800.0,0.637589
3915012,ZTS,2022-12-07,149.380005,153.789993,152.960007,150.250000,2444100.0,0.647193
3915013,ZTS,2022-12-08,149.199997,154.350006,150.529999,153.679993,2267500.0,0.643893
3915014,ZTS,2022-12-09,152.740005,156.330002,153.940002,153.389999,3274900.0,0.660068


In [5]:
history['CloseDiff'] = history.Close - history.Close.shift(1)
history['WeekDiff'] = history.Close.shift(1) - history.Close.shift(7)
history = history.fillna(0)
history

,Symbol,Date,Low,High,Open,Close,Volume,VolumeScaledNormalized,CloseDiff,WeekDiff
0,A,1999-11-18,28.612303,35.765381,32.546494,31.473534,62546380.0,0.789849,0.000000,0.000000
1,A,1999-11-19,28.478184,30.758226,30.713518,28.880545,15234146.0,0.727706,-2.592989,0.000000
2,A,1999-11-22,28.657009,31.473534,29.551144,31.473534,6577870.0,0.690754,2.592989,0.000000
3,A,1999-11-23,28.612303,31.205294,30.400572,28.612303,5975611.0,0.686529,-2.861231,0.000000
4,A,1999-11-24,28.612303,29.998213,28.701717,29.372318,4843231.0,0.677284,0.760015,0.000000
...,...,...,...,...,...,...,...,...,...,...
3915011,ZTS,2022-12-06,152.089996,155.500000,154.220001,153.050003,1964800.0,0.637589,-2.279999,5.229996
3915012,ZTS,2022-12-07,149.380005,153.789993,152.960007,150.250000,2444100.0,0.647193,-2.800003,4.919998
3915013,ZTS,2022-12-08,149.199997,154.350006,150.529999,153.679993,2267500.0,0.643893,3.429993,2.399994
3915014,ZTS,2022-12-09,152.740005,156.330002,153.940002,153.389999,3274900.0,0.660068,-0.289993,-0.460007


In [6]:
history['Sector'] = history['Symbol'].map(companies.set_index('Symbol')['Sector'])
history['Label'] = np.where(history['CloseDiff'] >= 0, 1, 0)
history

,Symbol,Date,Low,High,Open,Close,Volume,VolumeScaledNormalized,CloseDiff,WeekDiff,Sector,Label
0,A,1999-11-18,28.612303,35.765381,32.546494,31.473534,62546380.0,0.789849,0.000000,0.000000,Health Care,1
1,A,1999-11-19,28.478184,30.758226,30.713518,28.880545,15234146.0,0.727706,-2.592989,0.000000,Health Care,0
2,A,1999-11-22,28.657009,31.473534,29.551144,31.473534,6577870.0,0.690754,2.592989,0.000000,Health Care,1
3,A,1999-11-23,28.612303,31.205294,30.400572,28.612303,5975611.0,0.686529,-2.861231,0.000000,Health Care,0
4,A,1999-11-24,28.612303,29.998213,28.701717,29.372318,4843231.0,0.677284,0.760015,0.000000,Health Care,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3915011,ZTS,2022-12-06,152.089996,155.500000,154.220001,153.050003,1964800.0,0.637589,-2.279999,5.229996,Health Care,0
3915012,ZTS,2022-12-07,149.380005,153.789993,152.960007,150.250000,2444100.0,0.647193,-2.800003,4.919998,Health Care,0
3915013,ZTS,2022-12-08,149.199997,154.350006,150.529999,153.679993,2267500.0,0.643893,3.429993,2.399994,Health Care,1
3915014,ZTS,2022-12-09,152.740005,156.330002,153.940002,153.389999,3274900.0,0.660068,-0.289993,-0.460007,Health Care,0


In [7]:
history.to_csv('history_final.csv', index=False)